# lab 10d

## Notes

As far as I can tell, we really need to find two things:
- Trial Ion Product
    - To find this, we need to find the molarity of each element
    - To find molarity of each element, we need to find molarity of the compound
    - To find molarity of the compound, we need to find mols of compound
    - To find mols of compound, we need to convert grams to mols via `grams over molar mass`
    - So the pipeline is basically:
        - Take grams, convert to mols using $\frac{\text{1 mol}}{\text{(molar mass of X)g}}$, convert to molarity by dividing that by volume in L (convert that from mL), find molarity of each element by looking at the ratios, and then calculate reaction quotient using the proper exponents
- Solubility Product Constant
    - This is experimentally determined, isn't it?
    - We could use the grams per L solubility from PubChem...
        - essentially follow [this method](https://www.chemteam.info/Equilibrium/Calc-Ksp-FromMolSolub.html)
        - convert it to mols per L (molar solubility)
        - calculate $K_{sp}$ via reaction quotient formula
        - We [can't grab solubility programmatically](https://github.com/mcs07/PubChemPy/issues/16) though so it could be a pain
- Possibly useful: https://github.com/cgohlke/molmass

In [2]:
# Import necessary packages.
import pandas as pd

In [3]:
# Helper func for styling dataframes.
def style_df(df, caption=''):
    return df.style\
            .set_caption(caption)\
            .hide_index()\
            .format(precision=2)

In [4]:
raw_ss1 = pd.read_csv('solutionset-1.csv')
raw_ss2 = pd.read_csv('solutionset-2.csv')
raw_ss3 = pd.read_csv('solutionset-3.csv')
solutionset_1 = style_df(raw_ss1, caption='Solution Set I')
solutionset_2 = style_df(raw_ss2, caption='Solution Set II')
solutionset_3 = style_df(raw_ss3, caption='Solution Set III')
pubchem_elem = pd.read_csv('pubchem-elements.csv')

In [5]:
solutionset_1

Stock solutions,Mass,Volume
$Cu(NO_3)_2$,0.87,30.86
$CuCl_2$ anhydrous,0.50,31.18
$K_2CO_3$,0.71,42.70
$NaNO_3$,0.52,29.69
$KCl$,0.45,30.45


In [6]:
solutionset_2

Stock solutions,Mass,Volume
Na_2SO_4 * 10H_2O,1.16,29.75
$Sr(NO_3)_2$,0.91,0.00
$Ba(NO_3)_2$,0.80,30.53
$AlCl_3$,0.86,30.54
$K_2SO_4$,1.11,30.60


In [7]:
solutionset_3

Stock solutions,Mass,Volume
Fe(NO_3)_3 * 9H_2O,1.25,29.90
$Co(NO_3)_2$,1.12,30.48
$CoCl_2$,0.70,30.24
$NaOH$,0.25,30.10
$KOH$,0.34,30.00
$NaNO_3$,0.50,29.87


In [9]:
pubchem_elem

,AtomicNumber,Symbol,Name,AtomicMass,CPKHexColor,ElectronConfiguration,Electronegativity,AtomicRadius,IonizationEnergy,ElectronAffinity,OxidationStates,StandardState,MeltingPoint,BoilingPoint,Density,GroupBlock,YearDiscovered
0,1,H,Hydrogen,1.008000,FFFFFF,1s1,2.20,120.0,13.598,0.754,"+1, -1",Gas,13.81,20.28,0.000090,Nonmetal,1766
1,2,He,Helium,4.002600,D9FFFF,1s2,NaN,140.0,24.587,NaN,0,Gas,0.95,4.22,0.000179,Noble gas,1868
2,3,Li,Lithium,7.000000,CC80FF,[He]2s1,0.98,182.0,5.392,0.618,+1,Solid,453.65,1615.00,0.534000,Alkali metal,1817
3,4,Be,Beryllium,9.012183,C2FF00,[He]2s2,1.57,153.0,9.323,NaN,+2,Solid,1560.00,2744.00,1.850000,Alkaline earth metal,1798
4,5,B,Boron,10.810000,FFB5B5,[He]2s2 2p1,2.04,192.0,8.298,0.277,+3,Solid,2348.00,4273.00,2.370000,Metalloid,1808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,114,Fl,Flerovium,290.192000,NaN,[Rn]7s2 7p2 5f14 6d10 (predicted),NaN,NaN,NaN,NaN,"6, 4,2, 1, 0",Expected to be a Solid,NaN,NaN,NaN,Post-transition metal,1998
114,115,Mc,Moscovium,290.196000,NaN,[Rn]7s2 7p3 5f14 6d10 (predicted),NaN,NaN,NaN,NaN,"3, 1",Expected to be a Solid,NaN,NaN,NaN,Post-transition metal,2003
115,116,Lv,Livermorium,293.205000,NaN,[Rn]7s2 7p4 5f14 6d10 (predicted),NaN,NaN,NaN,NaN,"+4, +2, -2",Expected to be a Solid,NaN,NaN,NaN,Post-transition metal,2000
116,117,Ts,Tennessine,294.211000,NaN,[Rn]7s2 7p5 5f14 6d10 (predicted),NaN,NaN,NaN,NaN,"+5, +3, +1, -1",Expected to be a Solid,NaN,NaN,NaN,Halogen,2010


## Results
- tables of actual data we collected
- tables w calculations comparing reaction quotients to $K_{sp}$s
- summary of what was obtained, basically

## Discussion

- hypothesis => hootonian intervention required
- results significance => how the results relate back to core point of lab ($K_{sp}$s? solutbility?)
- scientific explanation => calculate $K_{sp}$s to explain why things played out the way they did
- error analysis => basically saying visual/contaimination/temp, and mostly saying they're negligible